In [ ]:
# Step 1: Mount Google Drive to access the dataset.
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os
import json
from datasets import Dataset, DatasetDict, Features, Value, Image as HFImage
from PIL import Image


In [ ]:
import os
import json
from datasets import Dataset, Features, Value, Image, concatenate_datasets

# Define paths to the image folders and caption files
image_folder1 = '/content/drive/MyDrive/Bangla Image dataset with caption/BNATURE/Pictures'
caption_file1 = '/content/drive/MyDrive/Bangla Image dataset with caption/BNATURE/caption/captions.json'
image_files_key1 = 'caption_id'
caption_key1 = 'bengali_caption'

image_folder2 = '/content/drive/MyDrive/Bangla Image dataset with caption/Bangla Lekha 2.0/images'
caption_file2 = '/content/drive/MyDrive/Bangla Image dataset with caption/Bangla Lekha 2.0/captions.json'
image_files_key2 = 'filename'
caption_key2 = 'caption'

image_folder3 = '/content/drive/MyDrive/Bangla Image dataset with caption/Flickr8k_Dataset/Flicker8k_Dataset'
caption_file3 = '/content/drive/MyDrive/Bangla Image dataset with caption/Flickr8k_Dataset/BAN-Cap_captiondata.json'
image_files_key3 = 'caption_id'
caption_key3 = 'bengali_caption'

# Function to load a single dataset
def load_dataset(image_folder, caption_file, image_files_key, caption_key):
    # Read captions file
    with open(caption_file, 'r', encoding='utf-8') as f:
        captions_data = json.load(f)

    # Create lists for images and captions
    images, captions = [], []
    for item in captions_data:
        image_file = os.path.join(image_folder, item[image_files_key])
        caption = item[caption_key]
        if os.path.exists(image_file):  # Ensure the image file exists
            images.append(image_file)
            captions.append(caption)

    # Return as a dictionary
    return {"image": images, "caption": captions}

# Load individual datasets into dictionaries
data1 = load_dataset(image_folder1, caption_file1, image_files_key1, caption_key1)
data2 = load_dataset(image_folder2, caption_file2, image_files_key2, caption_key2)
data3 = load_dataset(image_folder3, caption_file3, image_files_key3, caption_key3)

# Merge all datasets into a single dictionary
all_images = data1["image"] + data2["image"] + data3["image"]
all_captions = data1["caption"] + data2["caption"] + data3["caption"]

# Define features for the dataset
features = Features({
    "image": Image(),  # Explicitly define the Image feature
    "caption": Value("string")
})

# Create a combined dataset
combined_dataset = Dataset.from_dict({"image": all_images, "caption": all_captions}, features=features)

# Save combined dataset
combined_dataset.save_to_disk("bangla_combined_image_caption_dataset")
# Optionally: Push to Hugging Face Hub
# combined_dataset.push_to_hub("your-username/bangla-image-caption-dataset")


Saving the dataset (0/4 shards):   0%|          | 0/48259 [00:00<?, ? examples/s]

In [ ]:
import zipfile
from google.colab import files
# Save combined dataset to disk
dataset_dir = "bangla_combined_image_caption_dataset"
combined_dataset.save_to_disk(dataset_dir)
print(f"Dataset saved to {dataset_dir}")

# Compress the dataset into a zip file
zip_path = f"{dataset_dir}.zip"
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(dataset_dir):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, dataset_dir)
            zipf.write(file_path, arcname)
print(f"Dataset compressed into {zip_path}")


Saving the dataset (0/4 shards):   0%|          | 0/48259 [00:00<?, ? examples/s]

Dataset saved to bangla_combined_image_caption_dataset
Dataset compressed into bangla_combined_image_caption_dataset.zip


In [ ]:
from datasets import Dataset, Features, Value, Image
import os
import shutil

# Define paths
dataset_dir = "/content/bangla_combined_image_caption_dataset"  # Working directory for saving the dataset
gdrive_dir = "/content/drive/MyDrive/Bangla Image dataset with caption"  # Destination directory in Google Drive
zip_path = os.path.join(gdrive_dir, "bangla_combined_image_caption_dataset.zip")  # Final ZIP file path in Google Drive

# Save dataset to disk
print("Saving dataset to disk...")
combined_dataset.save_to_disk(dataset_dir)
print(f"Dataset saved to {dataset_dir}")

# Compress the dataset into a zip file
print("Compressing the dataset...")
shutil.make_archive(dataset_dir, 'zip', dataset_dir)
local_zip_path = f"{dataset_dir}.zip"  # Compressed file path in the local Colab environment
print(f"Dataset compressed to {local_zip_path}")

# Move the ZIP file to Google Drive
print("Moving the ZIP file to Google Drive...")
shutil.move(local_zip_path, zip_path)
print(f"Dataset successfully moved to Google Drive at {zip_path}")


Saving dataset to disk...


Saving the dataset (0/4 shards):   0%|          | 0/48259 [00:00<?, ? examples/s]

Dataset saved to /content/bangla_combined_image_caption_dataset
Compressing the dataset...
Dataset compressed to /content/bangla_combined_image_caption_dataset.zip
Moving the ZIP file to Google Drive...
Dataset successfully moved to Google Drive at /content/drive/MyDrive/Bangla Image dataset with caption/bangla_combined_image_caption_dataset.zip


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `jwdhashihisi` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenti

In [10]:
import os
import shutil
from huggingface_hub import HfApi

# Define paths
dataset_dir = "/content/bangla_combined_image_caption_dataset"
zip_path = f"{dataset_dir}.zip"

# Compress the dataset into a zip file
print("Compressing the dataset...")
shutil.make_archive(dataset_dir, 'zip', dataset_dir)
print(f"Dataset compressed to {zip_path}")

# Initialize Hugging Face API
api = HfApi()

# Push the ZIP file to the Hugging Face Hub
print("Uploading the ZIP file to Hugging Face Hub...")
api.upload_file(
    path_or_fileobj=zip_path,  # Path to the local ZIP file
    path_in_repo="bangla_combined_image_caption_dataset.zip",  # Path inside the dataset repository
    repo_id="Mansuba/bangla-image-caption-dataset",  # Repository name on Hugging Face Hub
    repo_type="dataset"  # Type of repository (dataset in this case)
)
print(f"Dataset successfully uploaded to Hugging Face Hub: Mansuba/bangla-image-caption-dataset")


Compressing the dataset...
Dataset compressed to /content/bangla_combined_image_caption_dataset.zip
Uploading the ZIP file to Hugging Face Hub...


No files have been modified since last commit. Skipping to prevent empty commit.


Dataset successfully uploaded to Hugging Face Hub: Mansuba/bangla-image-caption-dataset
